<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_AudioLDM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title Install AudioLDM2 Dependencies
import subprocess, os, sys, ipykernel

!pip install -q condacolab
import condacolab
condacolab.install()

res = subprocess.run(['conda', 'create', '-n' 'audioldm', 'python=3.8'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

res = subprocess.run(['conda', 'activate', 'audioldm'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

res = subprocess.run(['pip', 'install', 'audioldm'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(res)

# res = subprocess.run(['pip', 'install', 'git+https://github.com/haoheliu/AudioLDM2.git'], stdout=subprocess.PIPE).stdout.decode('utf-8')
# print(res)

#gitclone('https://github.com/haoheliu/AudioLDM')

print('install complete')








In [18]:
#@title Generate Audio


!apt-get install ffmpeg

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath, utils

import subprocess, os, sys, ipykernel
import random

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "4fc47a82-43c7-456a-9e0f-81c0bf4f128f" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()


# model_path = os.path.join(
#             os.path.expanduser("~"),
#             ".cache/audioldm/audioldm-l-full.ckpt",
#         )

# print('MODEL EXISTS:', str(os.path.exists(model_path)))




outputs_per_input = 3
batch_size = 1
candidates = 4
transfer_style = 'Romantic'
transfer_strength = '0.5'
output_dir = '/transfer_output'

def collect_outputs(input_file, output_dir):
    # Extract the file name without extension
    file_name = os.path.basename(input_file)
    file_name_without_ext = os.path.splitext(file_name)[0]

    # Create the new file path
    new_file_path = os.path.join(output_dir, "transfer", file_name_without_ext)

    # Check if the directory exists, if not create it
    if not os.path.exists(os.path.dirname(new_file_path)):
        os.makedirs(os.path.dirname(new_file_path))

    # Assuming the function does not actually transfer or create files,
    # we simulate by listing files in the given directory if they exist.
    # In a real scenario, this part would list files after transfer.
    file_paths = []
    if os.path.exists(new_file_path):
        for file in os.listdir(new_file_path):
            file_paths.append(os.path.join(new_file_path, file))

    return file_paths

async def dawnet_func(input_file: DAWNetFilePath):

  try:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    seed = str(random.randint(1, 1000))

    #audio_len=utils.get_audio_length(input_file)

    audio_len=10.0

    print("audio_length: " + str(audio_len))

    bpm_str = str(round(dawnet.get_daw_bpm()))
    print("bpm: " + bpm_str)

    print('INPUT_PATH: ' + str(input_file))
    print('OUTPUT_DIR: ' + str(output_dir))
    print('AUDIO_LEN: ' + str(audio_len))
    print('SEED: ' + seed)
    print('TRANSFER_STYLE: ' + transfer_style)
    print('BATCH SIZE: ' + str(batch_size))
    print('CANDIDATES: ' + str(candidates))

    print('RUN-START')

    #audioldm-l-full
    cmd = "audioldm --mode 'transfer' --file_path '{}' --save_path '{}' -t '{}' --transfer_strength {} --duration {} --seed {} --n_candidate_gen_per_text {} --batchsize {} --model_name audioldm-l-full".format(input_file, output_dir, transfer_style, transfer_strength, audio_len, seed, candidates, batch_size)

    !{cmd}

    print(res)

    outputs = collect_outputs(input_file, output_dir)
    for output in outputs:
        await dawnet.output().add_file(output)

    await dawnet.output().add_message("Returning the input file")
    await dawnet.output().send()

    return True

  except Exception as e:
    print(f"Error in arbitrary_method: {e}")
    return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("AudioLDM2 - Style Transfer")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()








# from IPython.display import Audio
# from IPython.display import display
# import csv
# import wave
# import contextlib
# import random


# mount_drive = '/content/drive/MyDrive/SNS_DIFFUSION'



# print('MODEL EXISTS', str(os.path.exists(model_path)))


# outputs_per_input = 10
# batch_size = 1
# candidates = 6
# transfer_styles = ['Energetic','Romantic','Melancholic','Nostalgic','Dreamy','Intense','Mysterious','Dark','Playful','Happy','Joyful','Sad','Hopeful','Excited','Sentimental','Calm','Peaceful','Serene','Relaxing','Soothing','Inspirational','Empowering','Motivational','Triumphant','Ethereal','Haunting','Enchanting','Passionate','Soulful','Sensual','Sexy','Funky','Groovy','Jazzy','Bluesy','Rocking','Driving','Explosive','Dynamic','Chill','Laid-back','Whimsical','Quirky','Eccentric','Wistful','Thoughtful','Contemplative','Reflective','Optimistic','Pensive','Fierce','Bold','Majestic','Grand','Regal','Epic','Cinematic','Trippy','Psychedelic','Futuristic','Industrial','Eerie','Creepy','Suspenseful','Tense','Edgy','Menacing','Aggressive','Angry','Frustrated','Resentful','Bittersweet','Disillusioned','Discontent','Disappointed','Conflicted','Desperate','Dramatic','Emotional','Fateful','Heartfelt','Heartbroken','Lonely','Nervous','Paranoid','Rebellious','Sentimental','Skeptical','Stressed','Suspicious','Tender','Tragic','Unsettling','Wary','Yearning','Zealous','Ambitious','Arrogant','Confident','Proud','Upbeat pop','Ethereal electronic music','Heavy metal guitar','Soothing melodies','Driving techno beats','Tender acoustic guitar','Catchy indie rock','Soulful jazz','Beautiful orchestral score','Funky bassline','Haunting ballad','Powerful hip-hop lyrics','Catchy hooks','Sultry vocals','Dreamlike atmosphere','Bluesy guitar solo','Harmonized a cappella','Experimental soundscapes','Atmospheric post-rock','Lyrical folk music','Rhythmic dance beats','Emotional piano ballads','Trippy psychedelic rock','Cheesy 80s pop','Jazzy saxophone solos','Groovy basslines','Fast-paced punk rock','Funky horn section','Soulful backing vocals','Nostalgic 90s hits','Experimental electronic beats','Epic movie soundtracks','Haunting choir vocals','Acoustic singer-songwriter','Hypnotic trance music','Melancholic indie pop','Aggressive rap verses','Atmospheric ambient music','Classic rock guitar riffs','Romantic love songs','Lively Caribbean rhythms','Percussive African beats','Energetic ska music','Patriotic military marches','Bittersweet acoustic ballads','Laid-back reggae vibes','Atmospheric post-punk','Moody synthpop','Spooky Halloween music','Uplifting motivational songs','Dreamy shoegaze guitar','Funky disco beats','Melodic alternative rock','Cheeky novelty songs','Hard-hitting drum and bass','Soulful Motown-inspired tunes','Ethereal choral music','Moody trip-hop','Latin-inspired salsa music','Calming new age music','Bright and sunny pop','Catchy synthwave tunes','Emotional country ballads','Angry punk anthems','Dreamlike indie folk','Slick hip-hop beats','Nostalgic retro sounds','Atmospheric black metal','Bouncy bubblegum pop','Heavy industrial beats','Uplifting gospel music','Progressive rock epics','Catchy Britpop anthems','Nostalgic 2000s hits','Soulful blues guitar','Soaring power ballads','Hypnotic sitar music','Intense heavy rock','Trippy ambient techno','Dreamy indie pop','Jazzy piano solos','Lively mariachi music','Catchy synthpop','Epic cinematic orchestration','Haunting piano melodies','Experimental noise music','Joyful Christmas carols','Upbeat disco funk','Melodic power metal','Cheesy boy band pop','Reflective folk rock','Groovy funk rock','Emotional post-hardcore','Bouncy ska punk','Patriotic American folk songs','Smooth R&B vocals','Atmospheric trip-hop beats','Upbeat electro swing','Catchy garage rock','Energetic dancehall reggae','Heavy doom metal','Heartfelt acoustic covers','Nostalgic synthpop','Piano','Guitar','Drums','Bass guitar','Violin','Cello','Trumpet','Saxophone','Clarinet','Flute','Harmonica','Accordion','Banjo','Mandolin','Ukulele','Harp','French horn','Trombone','Tuba','Oboe','Bassoon','Bagpipes','Didgeridoo','Bongos','Congas','Timpani','Maracas','Tambourine','Cowbell','Guiro','Djembe','Steel drums','Xylophone','Vibraphone','Marimba','Glockenspiel','Triangle','Chimes','Celesta','Organ','Synthesizer','Sampler','Theremin','Electronic drums','Electric guitar','Bass synthesizer','Double bass','Electric violin','Viola','Piccolo','Recorder','Bass clarinet','Euphonium','Sitar','Dulcimer','Zither','Autoharp','Hurdy-gurdy','Harpsichord','Clavichord','Mellotron','Fiddle','Mandola','Lute','Shamisen','Koto','Balalaika','Saz','Darbuka','Cymbals','Gong','Taiko drum','Rainstick','Ocean drum','Kalimba','Finger cymbals','Anklung','Jaw harp','Kalimba','Kazoo','Nose flute','Glass harmonica','Slide whistle','Swanee whistle','Bird whistle','Whamola','Theremin cello','Waterphone','Hulusi','Pungi','Crumhorn','Serpent','Sousaphone','Piccolo trumpet','Ocarina','Pan flute','Ney','Udu','Steel guitar','Chapman Stick','Acoustic','Alternative','Ambient','Americana','Avant-garde','Baroque','Big Band','Bluegrass','Blues','Bollywood','Bossa Nova','Celtic','Chamber','Chant','Choral','Classical','Country','Dance','Disco','Dixieland','Dream Pop','Drone','Dubstep','Electronic','Experimental','Flamenco','Folk','Funk','Gospel','Gothic','Grunge','Hard Rock','Heavy Metal','Hip Hop','House','Indie','Industrial','Jazz','Jungle','Klezmer','Latin','Lounge','Medieval','Metal','Minimalist','Motown','Musique Concrète','New Age','New Wave','Noise','Opera','Orchestral','Pop','Post Punk','Progressive','Psychedelic','Punk','R&B','Ragtime','Rap','Reggae','Renaissance','Rock','Rockabilly','Salsa','Samba','Shoegaze','Singer-Songwriter','Ska','Smooth Jazz','Soul','Southern Rock','Space Rock','Surf','Swing','Synth-pop','Techno','Trance','Trip Hop','Tropical','UK Garage','World','Worldbeat','Zouk','Afrobeat','Alternative Country','Balearic Beat','Barbershop','Bollywood Pop','Cumbia','Dancehall','Deep House','Electro Swing','Emo','Fado','Folktronica','Garage Rock','Glitch','Grindcore','Hardcore Punk']
# transfer_strength = '0.5'
# audio_files_dir = mount_drive + '/training/source_audio/sas_data/'
# output_dir = mount_drive + '/transfer_output'
# csv_path = mount_drive + '/transfer_runstate.csv'


# def print_files_in_directory(directory):
#     for root, dirs, files in os.walk(directory):
#         for file in files:
#             print(os.path.join(root, file))


# def audio_style_transfer(input_path: str, output_dir: str, audio_len:float) -> bool:

#     for _ in range(outputs_per_input):
#       seed = str(random.randint(1, 1000))
#       transfer_style = random.choice(transfer_styles)

#       print('INPUT_PATH: ' + str(input_path))
#       print('OUTPUT_DIR: ' + str(output_dir))
#       print('AUDIO_LEN: ' + str(audio_len))
#       print('SEED: ' + seed)
#       print('TRANSFER_STYLE: ' + transfer_style)
#       print('BATCH SIZE: ' + str(batch_size))
#       print('CANDIDATES: ' + str(candidates))

#       print('RUN-START')
#       try:
#         # env = os.environ.copy()
#         # res = subprocess.run(['audioldm','--mode','transfer','--file_path',"'" + input_path + "'",'--save_path', "'" + output_dir + "'",'-t',transfer_style,'--transfer_strength', str(transfer_strength),'--seed', str(seed)], shell=True, env=env).stdout.decode('utf-8')

#         cmd = "audioldm --mode transfer --file_path '{}' --save_path '{}' -t '{}' --transfer_strength {} --duration {} --seed {} --n_candidate_gen_per_text {} --batchsize {} --model_name audioldm-l-full".format(input_path, output_dir, transfer_style, transfer_strength, audio_len, seed, candidates, batch_size)

#         !{cmd}

#         print(res)
#       except Exception as e:
#         print('ERROR!!: ' + str(e))

#       print('RUN-END')

#       print("OUTPUT:")
#       print_files_in_directory(output_dir)

#       print("MODEL:")
#       print_files_in_directory(output_dir)

#       # res = subprocess.run(['audioldm', '--mode', 'transfer', '--file_path', input_path, '--save_path', output_dir,'-t', transfer_style,'--transfer_strength', transfer_strength,'--duration', str(audio_len),'--seed', seed], stdout=subprocess.PIPE).stdout.decode('utf-8')
#       # print(res)

#     return True

# def read_csv() -> list:
#     filenames = []
#     with open(csv_path, 'r') as f:
#         reader = csv.reader(f)
#         for y in reader:
#             if y[0] != '':
#                 filenames.append(y[0])

#     return filenames


# def write_csv(row_data):
#     with open(csv_path, mode='a') as my_csv:
#         my_csv_writer = csv.writer(my_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#         my_csv_writer.writerow([row_data])

# def has_file_been_processed(filename: str) -> bool:
#     if os.path.exists(csv_path) == False:
#         open(csv_path, 'w').close()

#     filenames = read_csv()
#     return filename in filenames




# for dir_name in os.listdir(audio_files_dir):
#     if "-drum" in dir_name:
#       print('SKIPPING DRUMS FOR NOW')
#       continue

#     dir_path = os.path.join(audio_files_dir, dir_name)
#     if os.path.isdir(dir_path) == False:
#         continue

#     for file_name in os.listdir(dir_path):
#         file_path = os.path.join(dir_path, file_name)
#         if os.path.isdir(file_path) == True:
#             continue

#         extension = os.path.splitext(file_name)[1]
#         if extension != '.wav':
#             continue

#         if has_file_been_processed(file_name):
#             print('FILE ALREADY PROCESSED: ' + str(file_name))
#         else:
#             print('PROCESSING: ' + str(file_path))
#             audio_len = 12.5
#             # with contextlib.closing(wave.open(file_path, 'r')) as f:
#             #     frames = f.getnframes()
#             #     rate = f.getframerate()
#             #     audio_len = frames / float(rate)
#             #     print(audio_len)

#             output_path = os.path.join(output_dir, dir_name)

#             if audio_style_transfer(file_path, output_path, float(audio_len)):
#                 write_csv(file_name)
#             else:
#                 print('ERROR: ' + str(file_name))






Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


2024-01-09 05:34:55,535 - websockets.client - DEBUG - = connection is CONNECTING
2024-01-09 05:34:55,538 - websockets.client - DEBUG - > GET / HTTP/1.1
2024-01-09 05:34:55,539 - websockets.client - DEBUG - > Host: 35.223.141.253:8765
2024-01-09 05:34:55,541 - websockets.client - DEBUG - > Upgrade: websocket
2024-01-09 05:34:55,542 - websockets.client - DEBUG - > Connection: Upgrade
2024-01-09 05:34:55,542 - websockets.client - DEBUG - > Sec-WebSocket-Key: G3xzV+7bwAcAVzo60Zb19g==
2024-01-09 05:34:55,543 - websockets.client - DEBUG - > Sec-WebSocket-Version: 13
2024-01-09 05:34:55,544 - websockets.client - DEBUG - > Sec-WebSocket-Extensions: permessage-deflate; client_max_window_bits
2024-01-09 05:34:55,548 - websockets.client - DEBUG - > User-Agent: Python/3.10 websockets/11.0.3
2024-01-09 05:34:55,583 - websockets.client - DEBUG - < HTTP/1.1 101 Switching Protocols
2024-01-09 05:34:55,584 - websockets.client - DEBUG - < Upgrade: websocket
2024-01-09 05:34:55,586 - websockets.client - 

audio_length: 10.0
bpm: 170
INPUT_PATH: /tmp/tmp4xe2ssf7/resampled/section_two-hi-0-7db83d0b-d5ce-4e4b-a028-6e042383a762.wav
OUTPUT_DIR: /transfer_output
AUDIO_LEN: 10.0
SEED: 529
TRANSFER_STYLE: Romantic
BATCH SIZE: 1
CANDIDATES: 4
RUN-START
Load AudioLDM: %s audioldm-l-full
DiffusionWrapper has 739.14 M params.
/usr/local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/usr/local/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be req

2024-01-09 05:37:25,869 - websockets.client - DEBUG - > TEXT '{"token": "4fc47a82-43c7-456a-9e0f-81c0bf4f128f...1-9100-7a4edb5e1560"}}}' [228 bytes]
2024-01-09 05:37:25,878 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_SEND_RESULTS_MSG', 'dn_msg': '{"token": "4fc47a82-43c7-456a-9e0f-81c0bf4f128f", "type": "results", "data": {"response": {"files": [], "error": null, "message": "Returning the input file", "status": "completed", "id": "22761c88-065d-4941-9100-7a4edb5e1560"}}}'}
2024-01-09 05:37:25,879 - DN_CLIENT - INFO - Processing event: {'dn_msg_stage': 'CLIENT_RUN_METHOD', 'dn_msg': 'Ran method: dawnet_func'}
2024-01-09 05:37:25,881 - websockets.client - DEBUG - < PING 3d ce 51 e1 [binary, 4 bytes]
2024-01-09 05:37:25,885 - websockets.client - DEBUG - > PONG 3d ce 51 e1 [binary, 4 bytes]
2024-01-09 05:37:25,886 - websockets.client - DEBUG - % sending keepalive ping
2024-01-09 05:37:25,887 - websockets.client - DEBUG - > PING 9b 4a da 92 [binary, 4 bytes]
2024-01-09 0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: torchlibrosa
    Found existing installation: torchlibrosa 0.1.0
    Uninstalling torchlibrosa-0.1.0:
      Successfully uninstalled torchlibrosa-0.1.0



2024-01-09 05:37:45,944 - websockets.client - DEBUG - % sending keepalive ping
2024-01-09 05:37:45,947 - websockets.client - DEBUG - > PING c2 8a 7b f4 [binary, 4 bytes]
2024-01-09 05:37:45,982 - websockets.client - DEBUG - < PONG c2 8a 7b f4 [binary, 4 bytes]
2024-01-09 05:37:45,984 - websockets.client - DEBUG - % received keepalive pong
2024-01-09 05:38:06,002 - websockets.client - DEBUG - % sending keepalive ping
2024-01-09 05:38:06,004 - websockets.client - DEBUG - > PING 39 ee 97 59 [binary, 4 bytes]
2024-01-09 05:38:06,039 - websockets.client - DEBUG - < PONG 39 ee 97 59 [binary, 4 bytes]
2024-01-09 05:38:06,040 - websockets.client - DEBUG - % received keepalive pong
2024-01-09 05:38:26,062 - websockets.client - DEBUG - % sending keepalive ping
2024-01-09 05:38:26,064 - websockets.client - DEBUG - > PING 27 ee 74 48 [binary, 4 bytes]
2024-01-09 05:38:26,099 - websockets.client - DEBUG - < PONG 27 ee 74 48 [binary, 4 bytes]
2024-01-09 05:38:26,100 - websockets.client - DEBUG - % r

KeyboardInterrupt: 